In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import morpher
import pandas as pd
import pandas_profiling
from collections import defaultdict
import math

In [ ]:
df_mimic = pd.read_csv("~/cohorts/hs_mimic.csv",index_col=0)
df_sinai = pd.read_csv("~/cohorts/hs_sinai.csv",index_col=0)

In [ ]:
xls = pd.ExcelFile('mapping.xls')
mapping = xls.parse(xls.sheet_names[1])
mapping.set_index('Field',inplace=True)

In [ ]:
df_sinai_copy = df_sinai.copy()

In [ ]:
df_sinai_copy.head()

In [ ]:
df_sinai[['hypertension_complicated_precondition_from_inf_days_before_to_0_days_after', 'hypertension_uncomplicated_precondition_from_inf_days_before_to_0_days_after']]

In [ ]:
merge_fields = defaultdict(lambda:[])
elixhauser = 0
series = defaultdict(lambda:[])

for _,row in df_sinai_copy.iterrows():
    for field in row.keys():        
        if mapping.loc[field,'Action'] == 'merge':
            merge_fields[mapping.loc[field,'Name']].append(row[field])
        if not pd.isna(mapping.loc[field,'Score']):
            elixhauser += int(row[field])*mapping.loc[field,'Score']            
    
    for mfield in merge_fields:
        series[mfield].append(any(merge_fields[mfield]))    
    series['ELIXHAUSER_SCORE'].append(elixhauser)
    
    merge_fields = defaultdict(lambda:[])
    elixhauser = 0    

for mfield in series:
    df_sinai_copy[mfield] = pd.Series(series[mfield])  
       

In [ ]:
df_sinai_copy[['HYPERTENSION']]

In [ ]:
for index, column in mapping.iterrows():
    if column['Action'] == 'remove' or column['Action'] == 'merge':
        df_sinai_copy.drop([index], axis=1, inplace=True)        
    if column['Action'] == 'rename':
        df_sinai_copy.rename(columns={ index : column['Name'] }, inplace=True)

In [ ]:
df_sinai_copy.head(26)

In [ ]:
df_sinai_copy_copy = df_sinai_copy.copy()

In [ ]:
df_sinai_copy_copy['AKI'].dtype
#df_sinai_copy_copy['HYPERTENSION'] = df_sinai_copy_copy['HYPERTENSION'].astype(bool)
#df_sinai_copy_copy['AKI'] = df_sinai_copy_copy['AKI'].astype(bool)

In [ ]:
for column in df_sinai_copy_copy:
    print(f"{column} is type { df_sinai_copy_copy[column].dtype }")


In [ ]:
df_sinai_copy_copy['AKI']

In [ ]:
for column in df_sinai_copy_copy:
    if df_sinai_copy_copy[column].dtype == bool:
        df_sinai_copy_copy[column] = df_sinai_copy_copy[column].apply(lambda x: x if x == np.NaN else int(x))
    


In [ ]:
df_sinai_copy_copy.head(26)

In [ ]:
df_sinai_copy_copy['AGE_AT_ADMISSION']

In [ ]:
df_sinai_copy_copy['AGE_AT_ADMISSION'] = df_sinai_copy_copy['AGE_AT_ADMISSION'].apply(lambda x: math.ceil(x/365))

In [ ]:
df_sinai_copy_copy['AGE_AT_ADMISSION'] 

In [ ]:
df_sinai_copy_copy['AGE_AT_ADMISSION'] = df_sinai_copy_copy['AGE_AT_ADMISSION'].apply(lambda x: x if x <= 90 else 90)

In [ ]:
df_sinai_copy_copy['AGE_AT_ADMISSION']

In [ ]:
df_sinai_copy_copy['GENDER']

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn_pandas import DataFrameMapper

In [ ]:
mapper = DataFrameMapper([('GENDER', LabelBinarizer())], df_out=True, default=None)

In [ ]:
mapper.fit_transform(df_sinai_copy_copy.copy())

In [ ]:
df_sinai_copy_copy.to_csv('~/cohorts/hs_sinai_preprocessed.csv',index=False)